### Navier-Stokes equations
We consider stationary Navier-Stokes on the unit square $\Omega = (0,1)^2$ with non-homogeneous Dirichlet boundary conditions ($u=(1,0)$) only on the top.

Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$, $\lambda \in \mathbb{R}$ so that
\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} + (\mathbf{u} \cdot \nabla) \mathbf{u} \cdot \mathbf{v}& - \int_{\Omega} \operatorname{div}(\mathbf{v}) p & &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q & & 
+ \int_{\Omega} \lambda q
&= 0 && \forall q \in Q, \\
& \int_{\Omega} \mu p & &= 0 && \forall \mu \in \mathbb{R}.
\end{align}


In [ ]:
from ngsolve.solvers import *
from ngsolve.meshes import *
from ngsolve import *
from draw import Draw
from netgen.geom2d import unit_square

In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))

In [ ]:
Re = Parameter(1)
V = VectorH1(mesh,order=2,dirichlet="bottom|right|top|left")
V.SetOrder(TRIG,3); V.Update()
Q = L2(mesh,order=1); 
N = NumberSpace(mesh); 
X = FESpace([V,Q,N])
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
a += (1/Re*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)
      -div(u)*q-div(v)*p-lam*q-mu*p)*dx

In [ ]:
gfu = GridFunction(X)
gfu.components[0].Set(CoefficientFunction((x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))

In [ ]:
Newton(a,gfu,maxit=20,dampfactor=1)

In [ ]:
Draw(0.25*gfu.components[1],mesh,"p")

In [ ]:
Draw(gfu.components[0],mesh,"u",sd=2)

In [ ]:
Re.Set(50)
Newton(a,gfu,dampfactor=0.5)
Draw(gfu.components[0],mesh,"u",sd=2)